In [1]:
!pip install fileupload

import fileupload
import re
import unicodedata
import pandas as pd
import operator
import json 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
def remove_url(str):
    str = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', str)
    return str
    
def remove_digit(str):
    str = re.sub(r'[^a-z ]*([.0-9])*\d', ' ', str)
    return str 

def remove_non_ascii(str):
    str = unicodedata.normalize('NFKD', str).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return str

def remove_twitter_char(str):  
    # mention
    str = re.sub(r'(?:@[\w_]+)', ' ', str)
    # hashtag
    str = re.sub(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", " ", str)
    # RT/cc
    str = re.sub('RT', ' ', str)

    return str

def remove_punctuation(str):
    str = re.sub(r'[^\s\w]', ' ', str)
    return str

def remove_multi_space(str):
    str = re.sub('[\s]+', ' ', str)
    return str

def casefolding(str):  
    str = str.lower()   
    return ' '.join(str.split())

def remove_repeated_character(str):
    str = re.sub(r'(.)\1{2,}', r'\1', str)
    return str

def normalize_slang_word(str):
    alay_dict = pd.read_csv('slang_word_list.csv', encoding='latin-1', header=None)
    alay_dict = alay_dict.rename(columns={0: 'original', 1: 'replacement'})
    alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in str.split(' ')])

def remove_laugh(str):
    str = re.sub(r"\b(?:(h|a|e)*(?:(ha|he|hue))+h?|(?:l+o+)+l+)|(?:(w|k)*(?:wk)+(w?|k?))\b", ' ', str)
    
    return str

def remove_unused_char(str):
    tmp = ""    
    if len(str) < 3:
        tmp
    else:
        tmp = str
        
    return tmp

In [3]:
def preprocessing(str):
    str = remove_url(str)
    str = remove_twitter_char(str)
    str = remove_digit(str)
    str = remove_non_ascii(str)
    str = remove_punctuation(str)
    str = remove_laugh(str)
    str = remove_multi_space(str) 
    str = remove_repeated_character(str)    
    str = casefolding(str)
    str = normalize_slang_word(str)
    str = remove_unused_char(str)
    
    return str

In [ ]:
data = pd.read_csv('train_preprocess1.csv')
sentence = data['sentence']
print(sentence)

# pada bagian ini nanti import ambil data dari database
# sesuai sql query
# select commentary from order_review
# where rating_model_sum >= 3 and id_freelancer == 
# "data yang diambil dari klik freelancer" 

0        warung ini dimiliki oleh pengusaha pabrik tahu...
1        mohon ulama lurus dan k mmbri hujjah partai ap...
2        lokasi strategis di jalan sumatra bandung temp...
3        betapa bahagia nya diri ini saat unboxing pake...
4        aduh jadi mahasiswa jangan sombong dong kasih ...
                               ...                        
10992                                         tidak kecewa
10993    enak rasa masakan nya apalagi kepiting yang me...
10994          hormati partai partai yang telah berkoalisi
10995    pagi pagi di tol pasteur sudah macet parah bik...
10996    meskipun sering belanja ke yogyakarta di riau ...
Name: sentence, Length: 10997, dtype: object


In [ ]:
sentence = sentence.apply(preprocessing)

In [ ]:
data2 = pd.read_csv('kata_sifat.csv')
adj = data2['word']
adj

0           abadi
1       abaktinal
2           abang
3        abjadiah
4        abnormal
          ...    
3591      zakelek
3592       zakiah
3593        zalir
3594      zig-zag
3595      zimotik
Name: word, Length: 3596, dtype: object

In [ ]:
count_word = {}
sentence = sentence.str.split(" ")

# print(sentences)

for sw in sentence:
  for i in sw:
    for sw2 in adj:
      if(i == sw2):
        count = count_word.get(i, 0)
        count += 1
        count_word[sw2] = count

In [ ]:
newA = dict(sorted(count_word.items(), key=operator.itemgetter(1), reverse=True)[:5])

In [ ]:
newA

{'cocok': 632, 'lain': 650, 'lebih': 792, 'nyaman': 932, 'salah': 520}

In [ ]:
part_nums = {'cocok': 632, 'lain': 650, 'lebih': 792, 'nyaman': 932, 'salah': 520}

In [ ]:
a = part_nums.keys()
b = part_nums.values() 

In [ ]:
list=[]
for idx1, i in enumerate(part_nums.keys()):
  for idx2, j in enumerate(part_nums.values()):
    if idx1 == idx2:
      list.append(i)
      list.append(j)

In [ ]:
list

['cocok', 632, 'lain', 650, 'lebih', 792, 'nyaman', 932, 'salah', 520]

In [ ]:
lst = []
for idx1, pn in enumerate(list):
  for idx2, i in enumerate(list):
    d = {}
    d['sifat']=pn
    if idx1 == idx2-1:
      d['value']=i
      lst.append(d)
      # hasil = json.dumps(d)
        
# json.dumps(lst)

In [ ]:
lst

[{'sifat': 'cocok', 'value': 632},
 {'sifat': 632, 'value': 'lain'},
 {'sifat': 'lain', 'value': 650},
 {'sifat': 650, 'value': 'lebih'},
 {'sifat': 'lebih', 'value': 792},
 {'sifat': 792, 'value': 'nyaman'},
 {'sifat': 'nyaman', 'value': 932},
 {'sifat': 932, 'value': 'salah'},
 {'sifat': 'salah', 'value': 520}]

In [ ]:
listAkhir = []

for idx1, pn in enumerate(lst):
  if idx1 % 2 == 0:
    listAkhir.append(pn)

In [ ]:
listAkhir

[{'sifat': 'cocok', 'value': 632},
 {'sifat': 'lain', 'value': 650},
 {'sifat': 'lebih', 'value': 792},
 {'sifat': 'nyaman', 'value': 932},
 {'sifat': 'salah', 'value': 520}]

In [ ]:
with open("hasil1.json", "w") as write_file:
    json.dump(listAkhir, write_file, indent=4)